In [ ]:
import pandas as pd
import psycopg2
import pyodbc
import datetime
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
#Passwords
ONEVIEW_PASSWORD = ''
EDW_USERNAME = ''
EDW_PASSWORD = ''
AIMS_USERNAME = ''
AIMS_PASSWORD = ''

In [ ]:
#OneView connection
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password= ONEVIEW_PASSWORD)

In [ ]:
#EDW connection
w = "DSN=prddw; UID={}; PWD={}".format(EDW_USERNAME, EDW_PASSWORD)
AMAEDW = pyodbc.connect(w)

In [ ]:
#AIMS connection
s = "DSN=aims_prod; UID={}; PWD={}".format(AIMS_USERNAME, AIMS_PASSWORD)
informix = pyodbc.connect(s)

In [ ]:
#Define universe
ov_me_sql = '''
    SELECT medical_education_number FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [ ]:
#get statistical area data
MSA_QUERY = \
    f"""
    SELECT 
    Z.ZIP,
    M.MSA_CD,
    M.POPULATION,
    M.MSA_TYPE,
    Z.CBSA,
    C.LEVEL_MSA,
    C.STATUS_1_OR_2
    FROM
    AMAEDW.CBSA_ZR C, AMAEDW.ZIP_CBSA_ZR Z, AMAEDW.MSA_ZR M
    WHERE
    C.ZIP_CBSA_DIVISION = Z.CBSA
    AND
    Z.ACTIVE_IND = 'Y'
    AND
    C.ACTIVE_IND = 'Y'
    AND
    C.CBSA_OR_DIV = 'CBSA'
    AND
    M.ACTIVE_IND = 'Y'
    """
statistic_area = pd.read_sql(con=AMAEDW, sql=MSA_QUERY)

In [ ]:
#get other census data
GEODATA_QUERY = \
    f"""
    SELECT 
    C.POPULATION AS CENSUS_GROUP,
    G.CENSUS_TRACT,
    G.BLOCK,
    G.FIPS_CD,
    G.ZIP_CD,
    G.ZIP_PLUS_4
    FROM
    AMAEDW.COUNTY_ZR C, AMAEDW.GEODATA_ZR G
    WHERE
    C.ACTIVE_IND = 'Y'
    AND
    G.FIPS_CD = C.FIPS_CD
    """
census = pd.read_sql(con=AMAEDW, sql=GEODATA_QUERY)

In [ ]:
#get fax data
FAX_QUERY = \
    """
    SELECT 
    P.PHONE_ID,
    P.PARTY_ID,
    N.AREA_CD,
    N.EXCHANGE,
    N.PHONE_NBR,
    P.FROM_DT,
    P.THRU_DT,
    P.CAT_CD_ID
    FROM
    AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.PARTY_PHONE P, AMAEDW.PHONE_NBR N
    WHERE
    C.PURPOSE_TYPE_ID = P.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD = 'F'
    AND
    P.PHONE_ID = N.PHONE_ID
    AND
    P.THRU_DT is null
    """
fax = pd.read_sql(con=AMAEDW, sql=FAX_QUERY)

In [ ]:
#get phone data
PHONE_QUERY = \
    """
    SELECT 
    P.PHONE_ID,
    P.PARTY_ID,
    N.AREA_CD,
    N.EXCHANGE,
    N.PHONE_NBR,
    P.FROM_DT,
    P.THRU_DT,
    P.CAT_CD_ID,
    C.PURPOSE_USG_CD
    FROM
    AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.PARTY_PHONE P, AMAEDW.PHONE_NBR N
    WHERE
    C.PURPOSE_TYPE_ID = P.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD = 'P'
    AND
    P.PHONE_ID = N.PHONE_ID
    AND
    P.THRU_DT is null
    """
phone = pd.read_sql(con=AMAEDW, sql=PHONE_QUERY)

In [ ]:
#get email data
EMAIL_QUERY = \
    """
    SELECT 
    P.EMAIL_ID,
    P.PARTY_ID,
    E.EMAIL_ADDR,
    E.EMAIL_STATUS,
    P.FROM_DT,
    P.THRU_DT,
    P.CAT_CD_ID,
    C.PURPOSE_USG_CD
    FROM
    AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.PARTY_EMAIL P, AMAEDW.EMAIL_ADDR E
    WHERE
    C.PURPOSE_TYPE_ID = P.PURPOSE_TYPE_ID
    AND
    P.EMAIL_ID = E.EMAIL_ID
    AND
    P.THRU_DT is null
    """
email = pd.read_sql(con=AMAEDW, sql=EMAIL_QUERY)

In [ ]:
#get party ids
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [ ]:
#get address data
POSTCD_QUERY = \
    f"""
    SELECT
    P.POST_CD_ID, 
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    P.STATE_ID,
    P.POST_CD,
    P.POST_CD_PLUS_4,
    P.LATITUDE,
    P.LONGITUDE,
    P.SRC_POST_KEY AS COMM_ID,
    FROM
    AMAEDW.POST_CD P
    """
post_cd = pd.read_sql(con=AMAEDW, sql=POSTCD_QUERY)

In [ ]:
#get party address relationship data
PARTY_ADDR_QUERY = \
    """
    SELECT 
    A.POST_CD_ID,
    A.PARTY_ID,
    A.FROM_DT,
    A.THRU_DT,
    A.CAT_CD_ID,
    C.PURPOSE_USG_CD,
    A.PARTY_ADDR_ID,
    C.PURPOSE_TYPE_CD,
    C.PURPOSE_USG_DESC
    FROM
    AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.PARTY_ADDR A
    WHERE
    C.PURPOSE_TYPE_ID = A.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD = 'A'
    AND
    A.THRU_DT is null
    AND
    C.SRC_SYS = 'MASTERFILE'
    """
party_addr = pd.read_sql(con=AMAEDW, sql=PARTY_ADDR_QUERY)

In [ ]:
#address undeliverable
UNDELIVERABLE_QUERY = \
    f'''
    SELECT
    ENTITY_ID,
    COMM_ID,
    COMM_EXC_CAT_CODE
    FROM
    informix.ENTITY_COMM_EXC_CT
    WHERE
    END_DT IS NULL
    ;
    '''
undeliverable = pd.read_sql(con=informix, sql=UNDELIVERABLE_QUERY)

In [ ]:
#get party level contact data
ids = pd.merge(party_ids, ov_me, left_on='ME', right_on='medical_education_number')
polos = party_addr[party_addr.PURPOSE_USG_CD=='PO  '][['PARTY_ID', 'POST_CD_ID']].drop_duplicates('PARTY_ID').rename(columns ={'POST_CD_ID':'POLO_ADDRESS'})
ppmas = party_addr[party_addr.PURPOSE_USG_CD=='PP  '][['PARTY_ID', 'POST_CD_ID']].drop_duplicates('PARTY_ID').rename(columns ={'POST_CD_ID':'PPMA_ADDRESS'})
addresses = party_addr[['PARTY_ID', 'POST_CD_ID']].drop_duplicates('PARTY_ID').rename(columns ={'POST_CD_ID':'ADDRESS'})
preferred_phones = phone[phone.PURPOSE_USG_CD=='PV  '][['PARTY_ID', 'PHONE_ID']].drop_duplicates('PARTY_ID').rename(columns ={'PHONE_ID':'PREFERRED_PHONE'})
phones = phone[['PARTY_ID', 'PHONE_ID']].drop_duplicates('PARTY_ID')
faxes = fax[['PARTY_ID', 'PHONE_ID']].drop_duplicates('PARTY_ID').rename(columns ={'PHONE_ID':'FAX_ID'})
preferred_emails = email[email.PURPOSE_USG_CD=='PE  '][['PARTY_ID', 'EMAIL_ID']].drop_duplicates('PARTY_ID').rename(columns ={'EMAIL_ID':'PREFERRED_EMAIL'})
emails = email[['PARTY_ID', 'EMAIL_ID']].drop_duplicates('PARTY_ID')

In [ ]:
#create party level table
PARTY_LEVEL = pd.merge(ids, addresses, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, ppmas, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, polos, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, phones, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, preferred_phones, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, emails, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, preferred_emails, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, faxes, on='PARTY_ID', how='left')

In [ ]:
#create comm level tables
PHONE = pd.merge(ids, phone, on='PARTY_ID')
FAX = pd.merge(ids, fax, on='PARTY_ID')
EMAIL = pd.merge(ids, email, on='PARTY_ID')
ADDRESS = pd.merge(ids, party_addr, on='PARTY_ID')
ADDRESS = pd.merge(ADDRESS, statistic_area, left_on='POST_CD', right_on ='ZIP_CD', how='left')
ADDRESS = pd.merge(ADDRESS, census, left_on=['POST_CD','POST_CD_PLUS_4'], right_on = ['ZIP_CD','ZIP_PLUS_4'], how='left')
ADDRESS = pd.merge(ADDRESS, undeliverable, on='COMM_ID', how='left')